# Excel

Mathematics is not just about *getting* a solution, but also about *communicating* it: clearly, precisely, in a manner and using a medium that your audience can understand.

When communicating a large amount of data, particularly to non-specialists, there is one tool that is widely known and used: Microsoft Excel. Whether you like it or loath it, at some point you'll probably have to use it.

Whilst you *can* program in Excel, here we'll focus on using Excel to communicate the results of other programs, or as a source of data.

## Middle-square method

We're going to generate some "random" numbers using Python. We'll then use Excel to see just how random they are. So our three step process is to

1. Write a Python code implementing the [Middle-square method](https://en.wikipedia.org/wiki/Middle-square_method) to generate random numbers;
2. Generate a lot of numbers and save them as an Excel workbook;
3. Analyze the results in Excel.

The Middle-square method takes a *seed*, which is an integer, with $n$ digits ($n$ must be even): for example, $23$ with $2$ digits. It squares the seed to get a number with $2 n$ digits: so $23^2 = 0529$ with $4$ digits. If necessary (as in this example), we "pad" the square with zeros to make the result have $2 n$ digits. Our new "random" number is found by taking the *middle* $n$ digits of the square: so the middle $2$ digits of $0529$ are $52$. To get the next "random" number we repeat the process: $52^2 = 2704 \to 70$, $70^2 = 4900 \to 90$, and so on.

Let's write a Python function that takes a seed and generates $N$ random numbers.

In [29]:
def middle_square(seed, N):
    """
    The Middle-square method for generating pseudo-random numbers.
    
    Parameters
    ----------
    
    seed : int
        The seed for the random number generation, n digits long
    N : int
        The number of random numbers to generate
        
    Returns
    -------
    
    random_list : list of int
        The random numbers: each will be an integer with n digits
    """
    
    n = len(str(seed))
    assert n%2 == 0, "n must be even"
    random_list = []
    previous_rn = seed
    for i in range(N):
        square = str(previous_rn**2)
        digits_square = len(square)
        square_padded = (2*n - digits_square) * '0' + square
        assert len(square_padded) == 2*n
        new_rn = int(square_padded[n-n//2:n+n//2])
        random_list.append(new_rn)
        previous_rn = new_rn
    return random_list

We can now test this on our simple seed, $23$.

In [30]:
print(middle_square(23, 10))

[52, 70, 90, 10, 10, 10, 10, 10, 10, 10]


We see this does a terrible job given this seed: it produces 4 numbers and then just repeats the last in a cycle. Does this always happen? We can use `set` to get the unique entries in the list, to count how well/badly it does:

In [38]:
for i in range(20, 40):
    print(i, len(set(middle_square(i, 15))))

20 2
21 8
22 4
23 4
24 2
25 5
26 5
27 5
28 5
29 4
30 2
31 10
32 2
33 4
34 6
35 5
36 5
37 6
38 8
39 4


Only one of these seeds produces more than a single-digit set of "random" numbers! Very poor.

However, life does improve as we increase the length of the seed:

In [39]:
print(middle_square(4269, 10))

[2243, 310, 961, 9235, 2852, 1339, 7929, 8690, 5161, 6359]


In [42]:
print(len(set(middle_square(4269,100))))

61


In [41]:
print(middle_square(41236797, 10))

[47342681, 32944426, 33520446, 62030003, 72127218, 33557641, 11526948, 87053019, 22811701, 37370251]


In [44]:
print(len(set(middle_square(41236797,10000))))

2364
